# Milestone 1: Basic model
In this chapter we will make a basic model that is able to predict the Pawpularity. 

In [ ]:
import pandas as pd
from os import chdir, listdir
import zipfile
import cv2

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
from tensorflow.keras.losses import MeanSquaredError

# Import zip with the data
The data is imported as a zip from the github of our project group. The zip is unpacked in the google colab, so the data is accesible. 

In [ ]:
# Code from: https://towardsdatascience.com/an-informative-colab-guide-to-load-image-datasets-from-github-kaggle-and-local-machine-75cae89ffa1e

# Get zip file from Github URL
!wget --no-check-certificate \
    "https://github.com/ilsefeenstra/Fitgirls0011/archive/refs/heads/main.zip" \
    -O "/tmp/pawpularity_data.zip"

# Opens the zip file in read mode and extract files into /tmp folder
zip_ref = zipfile.ZipFile('/tmp/pawpularity_data.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Import tabular data

The tabular data is imported. This contains information on whether several elements are present in the image, such as blur, a human, a group, etc. Also the pawpularity score of the training data is in the table. For the test data only the image ID and the features are in the table. There is also a sample submission table, which contains the pawpularity score for the test data.

In [ ]:
# Import the CSV tables
csv_train_data = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train.csv")
csv_test_data = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/sample_submission.csv")
csv_train_data.head()

# Drop rows with missing values (if NaN values are in dataframe)
# No missing values present, so no samples dropped
csv_train_data.dropna()

In [ ]:
# Create a plot that shows the distribution of the output of the training samples
plt.hist(csv_train_data['Pawpularity'], bins=100)
plt.title("Data distribution of the tabular data")
plt.xlabel("Pawpularity score")
plt.ylabel("Occurence")
plt.xlim(0, 100)

plt.show()

# Import image data
The images are imported from the folders. Each image is reshaped to a 64x64 image. In this way all the images have the same shape and we do not use much memory, to speed up analysis.


In [ ]:
def reshape_images(path, n):
    """
    This function returns a list of images, which are reshaped to 64 x 64 
    and a list with the names of the images.
    """
    # Set the current path
    chdir(path)
    
    # Preset the lists
    images = []
    image_names = []
    
    # Go over all the files in the path
    for i in listdir():
        
        # Get the name of the image, without .jpg
        image_names.append(i[:-4])
        
        # Get the image and reshape to n x n
        file = cv2.imread(i)
        file = cv2.resize(file,(n, n), interpolation=cv2.INTER_AREA)
        
        # Rescale the pixels and store in the list
        images.append(file/255)
        
    return images, image_names

# Reshape train and test images
train_imgs, train_names = reshape_images("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train", 64)
test_imgs, test_names = reshape_images("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/test", 64)

In [ ]:
# Make numpy arrays of images
train_imgs_array = np.array(train_imgs)
test_imgs_array = np.array(test_imgs)

# Combine tabular data with images
To ensure that the dataframe has the same order as the images in the list, we sort the dataframe based on the names of the images. If this would not be the case, it could be that you learn incorrectly, as the output of an image perhaps is not the real output.

In [ ]:
def sort_dataframe(data, images, names):
    """
    This function sorts the dataframe of the csv data according to the image names.
    """
    data_sorted = pd.DataFrame()

    # Iterate over images and get index of each image
    for img, name in zip(images, names):
        location = data[data['Id'] == name].index[0]

        # Sort dataframe according to index of images
        data_sorted = data_sorted.append([data.loc[location]])
        
    return data_sorted

# Sort training and testing data
train_data_sorted = sort_dataframe(csv_train_data, train_imgs, train_names)
test_data_sorted = sort_dataframe(csv_test_data, test_imgs, test_names)
sample_submission_sorted = sort_dataframe(sample_submission, test_imgs, test_names)

# Processing tabular data
Create seperate numpy arrays of the input and outputs.

In [ ]:
# Select x-values (the 12 input features) from training data
x_train_tabular = train_data_sorted.iloc[:,1:13].to_numpy()

# Select y-values (pawpularity) from training data
y_train = train_data_sorted.iloc[:,13].to_numpy()

# Select x-values (12 input features) from testing data
x_test_tabular = test_data_sorted.iloc[:,1:13].to_numpy()

# Select y-values (pawpularity) from testing data
y_test = sample_submission_sorted.iloc[:,1].to_numpy()

# Create seperate neural networks
We create a tabular neural network to handle the data in the csv. Then we create a convolutional neural network to handle the image data. Both neural networks have no output layer, since they will be concatenated to one neural network, which will give the output.

In [ ]:
def build_neural_net(input_size, hidden_nodes):
    """
    Build neural network with an input size and a hidden layer with a number of 
    hidden nodes.
    """
    # Create a sequential model object
    model = models.Sequential()

    # Create a hidden layer
    model.add(layers.Dense(units=hidden_nodes, activation='relu', input_shape=(input_size,)))
    
    return model

# Create neural network for tabular data and get summary of model 
# with 12 inputs and 100 hidden nodes
hidden_nodes = 20
tabular_NN = build_neural_net(12, hidden_nodes=20)
tabular_NN.summary()

In [ ]:
def build_convol_net(image_size, hidden_nodes):
    """
    Build neural network with an input size and a hidden layer with a number 
    of hidden nodes.
    """
    # Create a sequential model object
    model = models.Sequential()
    
    # Create a convolutional layer 
    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=image_size, padding='same'))
    
    # Create a maxpool layer
    model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    
    # Create a convolutional layer 
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))

    # Create a maxpool layer
    model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    
    # Create a flattening layer
    model.add(layers.Flatten())

    # Create a dense layer with relu activations
    model.add(layers.Dense(units=hidden_nodes, activation="relu"))
    
    return model

In [ ]:
# Create neural network for image data and get summary of model
image_size = (64, 64, 3)
image_NN = build_convol_net(image_size, hidden_nodes=20)
image_NN.summary()

## Concatenate tabular and image data models
Concatenate the tabular and image models to create one neural network that can handle both types of data. This neural network will give the prediction of the pawpularity.

In [ ]:
def concatenate_models(model1, model2, hidden_nodes):
    """
    Concatenate two neural network models, model1 and model2, and create
    a concatenated model with dense layers with some hidden nodes.
    """
    # Input for concatenated model is retrieved by concatenating the output
    # of both models
    concat_input = layers.concatenate([model1.output, model2.output])

    # Create hidden layers of the concatenated model
    hidden_layer_1 = layers.Dense(hidden_nodes, activation="relu")(concat_input)
    hidden_layer_2 = layers.Dense(hidden_nodes, activation="relu")(hidden_layer_1)
    output_layer = layers.Dense(1, activation="linear")(hidden_layer_2)

    # Create concatenated model with inputs of both models and output of the
    # concatenated model
    concat_model = models.Model(inputs=[model1.input, model2.input], outputs=output_layer)

    return concat_model

# Train the model
Train the concatenated model and plot the MSE of the training and validation data for each epoch.

In [ ]:
# Part of code from: https://www.tensorflow.org/tutorials/keras/regression

# Concatenate tabular and image neural networks
concat_model = concatenate_models(tabular_NN, image_NN, hidden_nodes=20)

# Compile model and use mean squared error as loss
concat_model.compile(loss=MeanSquaredError())

# Train the model by fitting both tabular and image data at the same time
history = concat_model.fit(x=[x_train_tabular, train_imgs_array], y=y_train, batch_size=20, epochs=20, validation_split=.2)

# Store loss during training in DataFrame
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

def plot_loss(history):
    """
    Plot loss during epochs of training a neural network.
    """
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

# Plot loss during training of concatenated model
plot_loss(history)

# See how the model scored
loss = concat_model.evaluate(x=[x_test_tabular, test_imgs_array], y=y_test)

# Print to 3 decimals
print(f'Test loss: {loss:.3}')

In [ ]:
# Create overview of layers in model
tf.keras.utils.plot_model(concat_model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')